In [1]:
import pyspark
from sklearn import preprocessing
import xgboost as xgb
import pandas as pd
import numpy as np
sc = pyspark.SparkContext('local[*]')

In [2]:
postulantes = pd.read_csv('Sources/postulantes_proc.csv').drop('Unnamed: 0', axis=1)
avisos = pd.read_csv('Sources/fiuba_6_avisos_detalle_limpio.csv').drop('Unnamed: 0', axis=1)
postulaciones = pd.read_csv('Sources/postulaciones_for_training.csv').drop('Unnamed: 0', axis=1)
visitas = pd.read_csv('Sources/visitas_proc.csv').drop('Unnamed: 0', axis=1)
test = pd.read_csv("Sources/test_final_100k.csv")

In [3]:
postulaciones = postulaciones.merge(avisos, on='idaviso')
postulaciones = postulaciones.merge(postulantes, on='idpostulante')
postulaciones = postulaciones.merge(visitas, on=['idaviso','idpostulante'], how='left')
postulaciones.drop(['titulo','descripcion'],axis=1, inplace=True)
postulaciones.head()

,idaviso,idpostulante,sepostulo,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,cluster,FEM,MASC,...,domingos,ingles,3 anos,2 anos,4 anos,5 anos,fechanacimiento,sexo,nivel,visito
0,1112428133,aRAV90,0.0,Full-time,Senior / Semi-Senior,Marketing,Bigbox,5.0,342.0,249.0,...,0,0,0,0,0,0,41.0,MASC,12,NaN
1,1111481489,aRAV90,0.0,Full-time,Senior / Semi-Senior,Atención al Cliente,RANDSTAD,216.0,NaN,NaN,...,0,0,0,0,0,0,41.0,MASC,12,NaN
2,1112389598,aRAV90,0.0,Full-time,Senior / Semi-Senior,Producción,Gentile y Asociados Consultores,106.0,49.0,1012.0,...,0,0,0,0,0,0,41.0,MASC,12,NaN
3,1112383515,aRAV90,0.0,Full-time,Senior / Semi-Senior,Calidad,Adecco - Spring Professional,155.0,27.0,168.0,...,0,0,0,0,0,0,41.0,MASC,12,NaN
4,1112278394,aRAV90,0.0,Full-time,Senior / Semi-Senior,Corporate Finance / Banca Inversión,Adecco -Región Cuyo & Patagonia,212.0,155.0,340.0,...,0,0,1,0,0,0,41.0,MASC,12,NaN


In [4]:
test = test.merge(avisos, on='idaviso')
test = test.merge(postulantes, on='idpostulante')
test = test.merge(visitas, on=['idaviso','idpostulante'], how='left')
test['sepostulo']=1
test.drop(['titulo','descripcion'],axis=1, inplace=True)
test.head()

,id,idaviso,idpostulante,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,cluster,FEM,MASC,...,ingles,3 anos,2 anos,4 anos,5 anos,fechanacimiento,sexo,nivel,visito,sepostulo
0,0,739260,6M9ZQR,Full-time,Jefe / Supervisor / Responsable,Comercial,BUMERAN.COM ARGENTINA,102.0,NaN,NaN,...,1,0,0,0,0,42.0,FEM,21,NaN,1
1,1,739260,6v1xdL,Full-time,Jefe / Supervisor / Responsable,Comercial,BUMERAN.COM ARGENTINA,102.0,NaN,NaN,...,1,0,0,0,0,31.0,MASC,9,NaN,1
2,2,739260,ezRKm9,Full-time,Jefe / Supervisor / Responsable,Comercial,BUMERAN.COM ARGENTINA,102.0,NaN,NaN,...,1,0,0,0,0,36.0,FEM,12,NaN,1
3,3,758580,1Q35ej,Full-time,Otro,Tecnologia / Sistemas,TeleTech,27.0,NaN,NaN,...,0,0,0,0,0,69.0,MASC,15,NaN,1
4,4,758580,EAN4J6,Full-time,Otro,Tecnologia / Sistemas,TeleTech,27.0,NaN,NaN,...,0,0,0,0,0,32.0,FEM,11,NaN,1


In [5]:
avisos = None
postulantes = None

In [6]:
resultado = postulaciones['sepostulo']
resultadob = test['sepostulo']
postulaciones.drop(['sepostulo','idaviso','idpostulante'],axis=1,inplace=True)
test.drop(['sepostulo','idaviso','idpostulante','id'],axis=1,inplace=True)


In [7]:
print(test.shape)
test.head()

(100000, 21)


,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,cluster,FEM,MASC,NO_DECLARA,TendenciaSexo,prom_edad,...,domingos,ingles,3 anos,2 anos,4 anos,5 anos,fechanacimiento,sexo,nivel,visito
0,Full-time,Jefe / Supervisor / Responsable,Comercial,BUMERAN.COM ARGENTINA,102.0,NaN,NaN,NaN,NaN,NaN,...,0,1,0,0,0,0,42.0,FEM,21,NaN
1,Full-time,Jefe / Supervisor / Responsable,Comercial,BUMERAN.COM ARGENTINA,102.0,NaN,NaN,NaN,NaN,NaN,...,0,1,0,0,0,0,31.0,MASC,9,NaN
2,Full-time,Jefe / Supervisor / Responsable,Comercial,BUMERAN.COM ARGENTINA,102.0,NaN,NaN,NaN,NaN,NaN,...,0,1,0,0,0,0,36.0,FEM,12,NaN
3,Full-time,Otro,Tecnologia / Sistemas,TeleTech,27.0,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,69.0,MASC,15,NaN
4,Full-time,Otro,Tecnologia / Sistemas,TeleTech,27.0,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,32.0,FEM,11,NaN


In [8]:
print(postulaciones.shape)
postulaciones.head()

(19783189, 21)


,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,cluster,FEM,MASC,NO_DECLARA,TendenciaSexo,prom_edad,...,domingos,ingles,3 anos,2 anos,4 anos,5 anos,fechanacimiento,sexo,nivel,visito
0,Full-time,Senior / Semi-Senior,Marketing,Bigbox,5.0,342.0,249.0,4.0,FEM,28.096610,...,0,0,0,0,0,0,41.0,MASC,12,NaN
1,Full-time,Senior / Semi-Senior,Atención al Cliente,RANDSTAD,216.0,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,41.0,MASC,12,NaN
2,Full-time,Senior / Semi-Senior,Producción,Gentile y Asociados Consultores,106.0,49.0,1012.0,31.0,MASC,30.400943,...,0,0,0,0,0,0,41.0,MASC,12,NaN
3,Full-time,Senior / Semi-Senior,Calidad,Adecco - Spring Professional,155.0,27.0,168.0,5.0,MASC,34.179487,...,0,0,0,0,0,0,41.0,MASC,12,NaN
4,Full-time,Senior / Semi-Senior,Corporate Finance / Banca Inversión,Adecco -Región Cuyo & Patagonia,212.0,155.0,340.0,12.0,MASC,38.044444,...,0,0,1,0,0,0,41.0,MASC,12,NaN


In [9]:
test = pd.concat([test,postulaciones])

In [10]:
visitas = pd.read_csv('Sources/visitas_proc.csv').drop('Unnamed: 0', axis=1)



In [11]:
test['fechanacimiento'].fillna(np.nanmean(test['fechanacimiento']),inplace=True)
test['FEM'].fillna(np.nanmean(test['FEM']),inplace=True)
test['MASC'].fillna(np.nanmean(test['MASC']),inplace=True)
test['NO_DECLARA'].fillna(np.nanmean(test['NO_DECLARA']),inplace=True)
test['prom_edad'].fillna(np.nanmean(test['prom_edad']),inplace=True)
test.loc[:,['sabados','domingos','ingles','3 anos','2 anos','4 anos','5 anos', 'visito']] = test.loc[:,['sabados','domingos','ingles','3 anos','2 anos','4 anos','5 anos', 'visito']].fillna(0)
test.head()

,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,cluster,FEM,MASC,NO_DECLARA,TendenciaSexo,prom_edad,...,domingos,ingles,3 anos,2 anos,4 anos,5 anos,fechanacimiento,sexo,nivel,visito
0,Full-time,Jefe / Supervisor / Responsable,Comercial,BUMERAN.COM ARGENTINA,102.0,795.729183,503.671695,19.340036,NaN,30.602718,...,0,1,0,0,0,0,42.0,FEM,21,0.0
1,Full-time,Jefe / Supervisor / Responsable,Comercial,BUMERAN.COM ARGENTINA,102.0,795.729183,503.671695,19.340036,NaN,30.602718,...,0,1,0,0,0,0,31.0,MASC,9,0.0
2,Full-time,Jefe / Supervisor / Responsable,Comercial,BUMERAN.COM ARGENTINA,102.0,795.729183,503.671695,19.340036,NaN,30.602718,...,0,1,0,0,0,0,36.0,FEM,12,0.0
3,Full-time,Otro,Tecnologia / Sistemas,TeleTech,27.0,795.729183,503.671695,19.340036,NaN,30.602718,...,0,0,0,0,0,0,69.0,MASC,15,0.0
4,Full-time,Otro,Tecnologia / Sistemas,TeleTech,27.0,795.729183,503.671695,19.340036,NaN,30.602718,...,0,0,0,0,0,0,32.0,FEM,11,0.0


In [12]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
for col in test.columns:
    if not test[col].dtype == 'O': continue
    print(col)
    test[col] = test[col].fillna('na')
    le.fit(test[col])
    test[col] = le.transform(test[col])
test.head()

tipo_de_trabajo
nivel_laboral
nombre_area
denominacion_empresa
TendenciaSexo
sexo


,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,cluster,FEM,MASC,NO_DECLARA,TendenciaSexo,prom_edad,...,domingos,ingles,3 anos,2 anos,4 anos,5 anos,fechanacimiento,sexo,nivel,visito
0,1,1,30,448,102.0,795.729183,503.671695,19.340036,2,30.602718,...,0,1,0,0,0,0,42.0,1,21,0.0
1,1,1,30,448,102.0,795.729183,503.671695,19.340036,2,30.602718,...,0,1,0,0,0,0,31.0,2,9,0.0
2,1,1,30,448,102.0,795.729183,503.671695,19.340036,2,30.602718,...,0,1,0,0,0,0,36.0,1,12,0.0
3,1,3,170,3809,27.0,795.729183,503.671695,19.340036,2,30.602718,...,0,0,0,0,0,0,69.0,2,15,0.0
4,1,3,170,3809,27.0,795.729183,503.671695,19.340036,2,30.602718,...,0,0,0,0,0,0,32.0,1,11,0.0


In [13]:
test = test.fillna(-1)
postulaciones = test[100000:]
test = test[:100000]

In [14]:
postulaciones.rename(columns={x:int(y) for x,y in zip(postulaciones.columns,range(1,len(postulaciones.columns)+1))},inplace=True)
test.rename(columns={x:int(y) for x,y in zip(test.columns,range(1,len(test.columns)+1))},inplace=True)

In [15]:
postulaciones.head()

,1,2,3,4,5,6,7,8,9,10,...,12,13,14,15,16,17,18,19,20,21
0,1,4,120,506,5.0,342.000000,249.000000,4.000000,0,28.096610,...,0,0,0,0,0,0,41.0,2,12,0.0
1,1,4,14,3157,216.0,795.729183,503.671695,19.340036,2,30.602718,...,0,0,0,0,0,0,41.0,2,12,0.0
2,1,4,143,1775,106.0,49.000000,1012.000000,31.000000,1,30.400943,...,0,0,0,0,0,0,41.0,2,12,0.0
3,1,4,25,203,155.0,27.000000,168.000000,5.000000,1,34.179487,...,0,0,0,0,0,0,41.0,2,12,0.0
4,1,4,44,204,212.0,155.000000,340.000000,12.000000,1,38.044444,...,0,0,1,0,0,0,41.0,2,12,0.0


In [16]:
test.head()

,1,2,3,4,5,6,7,8,9,10,...,12,13,14,15,16,17,18,19,20,21
0,1,1,30,448,102.0,795.729183,503.671695,19.340036,2,30.602718,...,0,1,0,0,0,0,42.0,1,21,0.0
1,1,1,30,448,102.0,795.729183,503.671695,19.340036,2,30.602718,...,0,1,0,0,0,0,31.0,2,9,0.0
2,1,1,30,448,102.0,795.729183,503.671695,19.340036,2,30.602718,...,0,1,0,0,0,0,36.0,1,12,0.0
3,1,3,170,3809,27.0,795.729183,503.671695,19.340036,2,30.602718,...,0,0,0,0,0,0,69.0,2,15,0.0
4,1,3,170,3809,27.0,795.729183,503.671695,19.340036,2,30.602718,...,0,0,0,0,0,0,32.0,1,11,0.0


In [17]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
#X_train, X_test, y_train, y_test = train_test_split(postulaciones, resultado, test_size=0.2)
clf=RandomForestClassifier(n_estimators=50,n_jobs=-1,warm_start=True,criterion='entropy')
clf.fit(postulaciones,resultado)
y_pred=clf.predict_proba(test)



In [18]:
test = pd.read_csv('Sources/test_final_100k.csv')
test.head()

,id,idaviso,idpostulante
0,0,739260,6M9ZQR
1,1,739260,6v1xdL
2,2,739260,ezRKm9
3,3,758580,1Q35ej
4,4,758580,EAN4J6


In [19]:
test['sepostulo'] = y_pred[:,1]
test.drop(['idaviso','idpostulante'],axis=1,inplace=True)
test.to_csv('Sources/resultados_random.csv', index=False)

In [20]:
y_pred[y_pred==1].shape

(72424,)

In [21]:
test.head()

,id,sepostulo
0,0,0.0
1,1,0.0
2,2,0.0
3,3,0.0
4,4,0.0
